# Exercises on Spark Streaming

The objective in this set of exercises is to get comfortable with streaming-style computations using Spark Streaming.

## First, an important warning

Because of the combination of the following elements, it will **not** be possible to solve this set of exercises inside this jupyter notebook. 

1. Streaming computations are computations that never finish (they continuously wait for new data to arrive).
2. We will need to run multiple computations in parallel (1 computation to generate data, 1 to consume data).
3. Jupyter does not allow cells to be executed in parallel.

**Instead** you will need to prepare individual python files, and execute these outside of jupyter. This notebook *only* serves as a guide of the exercises that you are requested to do.

## Next, a gentle reminder


![alt text](https://spark.apache.org/docs/latest/img/streaming-arch.png "")

Spark Streaming is a Spark library that allows streaming computations. 

In essence, Spark Streaming receives input data streams from any type of services (i.e: Kafka, Flume, HDFS, Kinesis, Twitter,...) and divides them into *mini-batches*. Those mini batches are then processed by Spark to build a final stream of results in batches. 

![alt text](https://spark.apache.org/docs/latest/img/streaming-flow.png "")

A continuous sequence of mini-batches is called a DStream. Each mini-batch in this DStream gets represented as an RDD and Spark Streaming provide a high level API that manipulate DStreams. 

![alt text](https://spark.apache.org/docs/latest/img/streaming-dstream.png) 

Any operation that is applied on a DStream translates to operations on the underlying RDDs. If we consider a simple example where the stream of data is a stream of lines of words (i.e: simple sentences for instance), a flatMap operation is applied on each RDD in the lines to generate as output the words DStreams containing a list of the words present in the processed sentence.

![alt text](https://spark.apache.org/docs/latest/img/streaming-dstream-ops.png) 

As already mentioned above, Spark Streaming can receive its input from different types of services, inlcuding Kafka, Twitter, Kinesis, ... . The simplest kind of streaming source, however, is the *file system*. In particular, when you set up Spark Streaming to receive data from a specific folder (which can be on your local filesystem, but could also be on HDFS), then it will watch this folder for new files to occur. Every new file will be treated as one mini-batch in the DStream. It is important to note that files that already existed in the watched folder when spark streaming starts will **not be processed**, only new files will be processed!

In this exercise session, we will use the file system as a source of streaming data. 

**But, see below for an example to connect to kafka**

The Spark Streaming programming guide is available at https://spark.apache.org/docs/latest/streaming-programming-guide.html

## 1. A simple Spark streaming example: counting inside a mini-batch

The `data` subfolder contains a file `data/orders.txt` that contains some historal data of buy and sell orders on a stock exchange. 

In [4]:
# shows the first 5 lines of orders.txt
import headtail
headtail.head('data/orders.txt', 5)

['2016-03-22 20:25:28,1,80,EPE,710,51.00,B\n',
 '2016-03-22 20:25:28,2,70,NFLX,158,8.00,B\n',
 '2016-03-22 20:25:28,3,53,VALE,284,5.00,B\n',
 '2016-03-22 20:25:28,4,14,SRPT,183,34.00,B\n',
 '2016-03-22 20:25:28,5,62,BP,241,36.00,S\n']

Each line has the following fields.
* Order timestamp—Format yyyy-mm-dd hh:MM:ss
* Order ID —Serially incrementing integer
* Client ID —Integer randomly picked from the range 1 to 100
* Stock symbol—Randomly picked from a list of 80 stock symbols
* Number of stocks to be bought or sold—Random number from 1 to 1,000
* Price at which to buy or sell—Random number from 1 to 100
* Character B or S —Whether the event is an order to buy or sell

The contents of ` data/orders.txt` is split into multiple files in the subfolder `data/split`. For example, `data/split/ordersaa.ordtmp` contains the first 1000 lines of `data/orders.txt`; `ordersab.ordtmp` contains the next 1000, and so on.

In [2]:
import headtail
headtail.head('data/split/ordersaa.ordtmp', 5)

['2016-03-22 20:25:28,1,80,EPE,710,51.00,B\n',
 '2016-03-22 20:25:28,2,70,NFLX,158,8.00,B\n',
 '2016-03-22 20:25:28,3,53,VALE,284,5.00,B\n',
 '2016-03-22 20:25:28,4,14,SRPT,183,34.00,B\n',
 '2016-03-22 20:25:28,5,62,BP,241,36.00,S\n']

In [3]:
import headtail
headtail.head('data/split/ordersab.ordtmp', 5)

['2016-03-22 20:25:28,10001,58,ABX,63,6.00,B\n',
 '2016-03-22 20:25:28,10002,21,X,761,45.00,S\n',
 '2016-03-22 20:25:28,10003,32,RDS.A,767,79.00,B\n',
 '2016-03-22 20:25:28,10004,56,UAL,904,30.00,S\n',
 '2016-03-22 20:25:28,10005,1,WLL,203,22.00,B\n']

The python script `scripts/simulateStreamingInput.py` can be used to simulate new data arriving in a streaming fashion. Concretely, it copies the files from `data/split` to  the folder `stream-IN` one by one, with a delay of 3 seconds in-between two files. If we start Spark streaming to monitor the `stream-IN` folder for new files, then the net effect is that every 3 seconds, 1000 lines of stock trade date is made available to Spark Streaming.

**Example 1.1.** The file `1-countPerBatch.py` creates a Spark Streaming job that monitors the folder `stream-IN`for new files. For each mini-batch (which contains the contents of these new files), it will parse each text line in mini-batch into a python dictionary. Next, it computes the total number of lines that contain a *Buy* 
order(last column = 'B') and the total number of lines that contain a *Sell* order (last column = 'F'). Finally, the first 10 lines of each mini-batch are printed on the console

**Exercise 1.2** Do the following.

1. Inspect the contents of the file `1-countPerBatch.py`. See if you understand what is being done
2. Execute this python script, either by `python 1-countPerBatch.py` (which will work if pyspark is correctly added to your PYTHONPATH variable), or by running `<path-to-spark>/bin/spark-submit 1-countPerBatch.py`. In parallel (i.e., in a separate shell/command line), execute `python scripts/simulateStreamingInput.py` to start copying data to the `stream-IN` folder.
3. You can terminate the Spark Streaming and `simulateStreamingInput` jobs by pressing control+C
4. Modify `1-countPerBatch.py` and uncomment the line that saves every RDD in the DStream to stream-OUT. Re-execute to see what happens

---
**Some comments**:

* Just like you need a SparkContext object to construct RDDs, you need a StreamingContext to construct DStreams. StreamingContext are created from an existing SparkContext.
* Only 1 StreamingContext can be executing per JVM, i.e., 1 per spark streaming job
* You can stop a StreamingContext `ssc` by calling `ssc.stop()`. This, however will also close the SparkContext that was used to create it. Call `ssc.stop(False)` to avoid closing the SparkContext (which can then be used to construct a new StreamingContext)
---


**Exercise 1.3** Copy `1-countPerBatch.py` into a file `1.3-countAndVolumePerBatch.py` and modify the latter to output, for each mini-batch, the following pairs:

```
('BUY', total number of buy orders in this minibatch RDD)
('SELL', total number of sell orders in this minibatch RDD)
('BUYVOL', total volume bought in this minibatch RDD)
('SELLVOL', total volume sold in this minibatch RDD)
```
Here, the *volume* of an order is the order's amount times the order's price.

(Hint: create two dstreams, one for the counts and one for the volumes, and union them  with the `union` method of dstreams).

Be sure to test your implementation.

**Exercise 1.4** Copy `1-countPerBatch.py` into a file `1.4-countAndVolumePerBatch.py` and modify the latter to output, for each mini-batch, the following pairs:
```
('BUY': total number of buy orders in this minibatch)
('SELL': total number of sell orders in this minibatch)
('<userid>': total volume traded (bought or sold) by this user-id in this mini-batch)
```
Where the last pair is repeated for every `<userid>` present in the current minibatch.

## 2. Aggregating data across mini-batches

Often we need to compute aggregates of data that spans multiple mini-batches. The file `2-totalVolumePerClient.py` will output, for each mini-batch, the following pairs:

```
('BUY': total number of buy orders in this minibatch RDD)
('SELL': total number of sell orders in this minibatch RDD)
('<userid>': total volume traded by this user-id across all mini-batches, present and past)
```
Where the last pair is repeated for every `<userid>` ever encountered and the total volume includes both buys and sells.

It works by using the `updateStateBykey` function of pair DStreams, which allows remember a state (per key) across minibatches. Concretely, updateStateByKey takes as argument function that gets two inputs: the set of new values for the key (in this minibatch) and the old state (which is `None` if the key hasn't been seen before). It needs to output the new state to be maintained. This also becomes part of the output RDD.

In our example, the state is just the current volume bought and sold, i.e., an integer.

**Exercise 2.1**
1. Inspect the contents of the file `2-totalVolumePerclient.py`. See if you understand what is being done
2. Execucte this python script. In parallel (i.e., in a separate shell/command line), execute `scripts/simulateStreamingInput.py` to start copying data to the `stream-IN` folder.
3. You can terminate the Spark Streaming and `simulateStreamingInput` jobs by pressing control+C

**Exercise 2.2** Copy `2-totalVolumePerClient.py` into a file `2.2-top5Clients.py` and 
modify the latter to output, for each mini-batch, the user ids of the top 5 clients (i.e., the 5 clients that have the largest buy/sell volume over all orders seen so far).

*Hint*: to calculate the top-5 elements of an RDD you can first sort the RDD (using `sortBy`) and then then take the first 5 elements (first `zipWithIndex` to associate the index to each element, then filter only those elements whose index is less than 5). Note, however, that a DStream is a sequence of RDDs, not a single RDD. So, you need to do this transformation on each rdd in the DStream, which you can do by means of the DStream's `transform()` method (which takes as argument a function that transforms the RDD).


## 3. Time-limited aggregates using windows

Using windowing operations, we can time-limited aggregates. 

**Example 3.1** An example is given in `3-salesPerMinutes.py`, which computes the total number of orders seen in the last minute, with a refresh of this total every 15 seconds.

**Exercise 3.2**  
1. Inspect the contents of the file `3-salesPerMinute.py`. See if you understand what is being done
2. Execucte this python script. In parallel (i.e., in a separate shell/command line), execute `scripts/simulateStreamingInput.py` to start copying data to the `stream-IN` folder. The totals reported should increased during the first minute, and then stabilize. Once stabilized, cancel the `simulateStreamingInput` script; the reported numbers should now start to decrease.
3. You can terminate the Spark Streaming and `simulateStreamingInput` jobs by pressing control+C

**Exercise 3.3** Copy `3-salesPerMinute.py` into a file `3.3-top5Securities.py` and 
modify the latter to compute the top five most traded securities in the last 3minutes, which is updated every 10 seconds.

## 4. A SparkStreaming Example that receives input from Kafka and outputs to Kafka

So far, we have been using Spark Streaming to read from the filesystem, and output to the console or the filesystem. In this final exercise, we will run a spark streaming job **on the cluster** that consumes input from Kafka.

Specifically:

1. Connect to the bigdata cluster
2. Clone this repository on the cluster by executing the following command in the SSH command line.

   ```bash
   git clone https://cs-devel.ulb.ac.be/rhodecode/Teaching/INFOH515-ex
   ```
   Subsequently, go the the cloned folder by executing `cd INFOH515-ex` on the SSH command line.
   
3. Create a new Kafka topic that will be used to receives stock quotes
   ```bash
   kafka-topics --create --zookeeper bigdata01.hpda.ulb.ac.be:2181 \
   --topic $USER.orders --partitions 5 --replication-factor 3
   ```      
   
4. The script `scripts/streamOrdersToKafka.py` can be used send the contents of `data/orders.txt` to the `$USER.orders` kafka topics, line by line, with 1 line published every 0.5 seconds. **Note** that you need to execute this script **on the bigdata cluster** in order for this to work!

5. Open a second SSH connection to the bigdata cluster. Go to the INFOH515-ex subfolder and execute `4-salesPerMinuteKafka.py` in the following way to compute the total number of orders in the last minute published in the `$USER.orders` topic. 
   ```bash
   spark-submit --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.0 4-salesPerMinuteFromKafka.py
   ```
  
The totals are printed on the console.

**Note:**: you need to run the python script by calling spark-submit because this will automatically download the spark-streaming-kafka JAR file on which the script depends.
